In [6]:
from snowflake.snowpark import Session


connection_parameters = {

     "account":"zfb96811.us-east-1", 
     "role":"ACCOUNTADMIN", 
     "database":"ANALYTICS", 
     "schema":"PUBLIC",
     "warehouse":"COMPUTE_WH",
    "user": "IMAHAKUMARA",
    "password":"gEfnun-dewnin-0nadpy"    
}

session = Session.builder.configs(connection_parameters).create()

In [28]:
# Test notebook
import pandas as pd
from datetime import date, timedelta

from snowflake.snowpark.functions import date_trunc, current_date
from snowflake.snowpark.functions import col, lit, when
from snowflake.snowpark.types import DateType

import snowflake.snowpark.functions as f


In [24]:
df = session.table("analytics.analytics_inference.skio_subscriptions").where(col('frequency')=='bi-monthly')

In [25]:
df.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CREATED_WEEK"  |"EMAIL"                              |"CUSTOMER_ID"  |"SUBSCRIPTIONID"                      |"FREQUENCY"  |"CREATEDAT"  |"CANCELLEDAT"  |"CANCELLED_WEEK"  |"CYCLESCOMPLETED"  |"STATUS"   |"STOREFRONTUSERID"                    |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2024-08-19      |lauren.f.hunt@gmail.com              |7021556498537  |1d33e54e-4a91-4340-8264-422f85834445  |bi-monthly   |2024-08-20   |NULL           |NULL              |5                  |ACTIVE     |d2138467-a2f9-469e-9ea3-1ec1ed667a5b  |
|2024-11-25     

In [10]:
today = date.today()
this_monday = today - timedelta(days=today.weekday())

mondays = [this_monday - timedelta(weeks=i) for i in range(8)]

mondays

[datetime.date(2025, 6, 2),
 datetime.date(2025, 5, 26),
 datetime.date(2025, 5, 19),
 datetime.date(2025, 5, 12),
 datetime.date(2025, 5, 5),
 datetime.date(2025, 4, 28),
 datetime.date(2025, 4, 21),
 datetime.date(2025, 4, 14)]

In [37]:
snapshot_week = date(2025, 5, 26)  # or however you're setting it
week_plus_14 = snapshot_week + timedelta(days=14)



churn_labeled_snapshot = (
    df.where(col("CREATED_WEEK")<snapshot_week).select(
        lit(snapshot_week).alias("snapshot_week"),
        col("SUBSCRIPTIONID"),
        col("email"),
        col("createdat"),
        col("cancelledat"),
        when(
            col("cancelledat").is_not_null(), f.datediff('week', col("CREATED_WEEK"), col("CANCELLEDAT"))).otherwise(
            f.datediff('week', col("CREATED_WEEK"), col("snapshot_week"))
        ).alias("lifetime"),
        when(
            (col("cancelledat").is_not_null()) &
            (col("cancelledat") <= lit(snapshot_week)),
            1
        ).otherwise(0).alias("snapshot_churn_state"),
          
    )
)

In [40]:
churn_labeled_snapshot.where(col('cancelledat').is_null()).show()


----------------------------------------------------------------------------------------------------------------------------------------------------------
|"SNAPSHOT_WEEK"  |"SUBSCRIPTIONID"                      |"EMAIL"                    |"CREATEDAT"  |"CANCELLEDAT"  |"LIFETIME"  |"SNAPSHOT_CHURN_STATE"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------
|2025-05-26       |9782f909-d05c-4993-9f4d-48ca59c4306c  |nancyjgregg@gmail.com      |2024-07-05   |NULL           |47          |0                       |
|2025-05-26       |3d7fe005-88ac-454b-b2d1-f2b042bd89f2  |nathalie@ndmossdesign.com  |2025-01-27   |NULL           |17          |0                       |
|2025-05-26       |92755da6-9bf4-4f2c-ac03-0b6f20c608ef  |ericaconnelly@hotmail.com  |2024-12-10   |NULL           |24          |0                       |
|2025-05-26       |aa4c85c5-6b7d-469e-afe0-8b1a6b985547  |bianchiolson